In [ ]:
import pandas as pd
from statsmodels.tsa.seasonal import seasonal_decompose
from pmdarima import auto_arima
import matplotlib.pyplot as plt


In [ ]:
df = pd.read_csv('cleaned_sales_data.csv')


In [ ]:
# Convert 'Month' to datetime objects and set as index
df['Month'] = pd.to_datetime(df['Month'])
df.set_index('Month', inplace=True)


In [ ]:
# Resample to monthly sales
monthly_sales = df['Sales'].resample('M').sum()

In [ ]:
# Time series decomposition
decomposition = seasonal_decompose(monthly_sales, model='additive')
fig = decomposition.plot()
fig.set_size_inches(12, 8)
plt.savefig('decomposition.png')
plt.close()


In [ ]:
# Fit auto_arima model
arima_model = auto_arima(monthly_sales, seasonal=True, m=12,
                         suppress_warnings=True, stepwise=True)

print(arima_model.summary())

In [ ]:
# Forecast for the next 12 months
forecast, conf_int = arima_model.predict(n_periods=12, return_conf_int=True)

In [ ]:
# Create a date range for the forecast
forecast_index = pd.date_range(start=monthly_sales.index[-1] + pd.DateOffset(months=1),
                               periods=12, freq='M')

In [ ]:
# Create a pandas Series for the forecast
forecast_series = pd.Series(forecast, index=forecast_index)
conf_int_df = pd.DataFrame(conf_int, index=forecast_index, columns=['lower', 'upper'])


In [ ]:
# Plot the forecast
plt.figure(figsize=(12, 6))
plt.plot(monthly_sales, label='Historical Sales')
plt.plot(forecast_series, label='Forecast')
plt.fill_between(forecast_index, conf_int_df['lower'], conf_int_df['upper'],
                 color='k', alpha=0.15)
plt.title('Sales Forecast')
plt.xlabel('Month')
plt.ylabel('Sales')
plt.legend()
plt.grid(True)
plt.savefig('sales_forecast.png')
plt.close()

# Print the forecast values
print("\nForecasted Sales for the Next 12 Months:")
print(forecast_series)